# Spacegroup determination: comparing spglib vs neural network based on the diffraction intensity in spherical harmonics (DISH) descriptor

Author: Angelo Ziletti (angelo.ziletti@gmail.com; ziletti@fhi-berlin.mpg.de)

In [3]:
from ai4materials.dataprocessing.preprocessing import load_dataset_from_file
from ai4materials.wrappers import load_descriptor
from ai4materials.utils.utils_config import set_configs
from ai4materials.utils.utils_config import setup_logger
from ase.spacegroup import get_spacegroup as ase_get_spacegroup
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
%matplotlib inline  

main_folder = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/'
dataset_folder = os.path.abspath(os.path.normpath(os.path.join(main_folder, 'datasets')))
desc_folder = os.path.abspath(os.path.normpath(os.path.join(main_folder, 'desc_folder')))

configs = set_configs(main_folder=main_folder)
logger = setup_logger(configs, level='INFO', display_configs=False)
configs['io']['dataset_folder'] = dataset_folder
configs['io']['desc_folder'] = desc_folder

## Displacements 0.4%

In [220]:
# hcp - spacegroup 194
filenames_disp04_hcp = ['hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid1_disp0004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp0004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp0004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid4_disp0004.tar.gz']
# sc - spacegroup 221
filenames_disp04_sc = ['sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp0004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid2_disp0004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp0004.tar.gz']

# fcc - spacegroup 225
filenames_disp04_fcc = ['fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz']

# diam - spacegroup 227
filenames_disp04_diam = ['diam/disp/A_cF8_227_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid1_disp0004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid3_disp0004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid4_disp0004.tar.gz']
# bcc - spacegroup 229
filenames_disp04_bcc = ['bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid1_disp0004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid2_disp0004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid4_disp0004.tar.gz']
                         
                         
desc_files_disp04_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp04_hcp]
desc_files_disp04_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp04_sc]
desc_files_disp04_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp04_fcc]
desc_files_disp04_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp04_diam]
desc_files_disp04_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp04_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_disp004_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_disp004_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_disp004_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_disp004_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_disp004_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a

INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF

INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp0004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/di

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp0004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp0004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI

In [235]:
structure_list_04 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_disp04 = []
for structure in structure_list_04:
    y_pred_disp04.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Displacements 1.0%

In [229]:
# hcp - spacegroup 194
filenames_disp1_hcp = ['hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid1_disp001.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp001.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp001.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid4_disp001.tar.gz']
# sc - spacegroup 221
filenames_disp1_sc = ['sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp001.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid2_disp001.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp001.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp001.tar.gz']

# fcc - spacegroup 225
filenames_disp1_fcc = ['fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz']

# diam - spacegroup 227
filenames_disp1_diam = ['diam/disp/A_cF8_227_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid1_disp001.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid3_disp001.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid4_disp001.tar.gz']
# bcc - spacegroup 229
filenames_disp1_bcc = ['bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid1_disp001.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid2_disp001.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid4_disp001.tar.gz']
                         
                         
desc_files_disp1_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp1_hcp]
desc_files_disp1_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp1_sc]
desc_files_disp1_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp1_fcc]
desc_files_disp1_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp1_diam]
desc_files_disp1_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp1_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_disp1_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_disp1_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_disp1_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_disp1_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_disp1_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp001.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_c

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp001.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp001.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

In [230]:
structure_list_1 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_disp1 = []
for structure in structure_list_1:
    y_pred_disp1.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Displacements 2.0%

In [236]:
# hcp - spacegroup 194
filenames_disp2_hcp = ['hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid1_disp002.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp002.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp002.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid4_disp002.tar.gz']
# sc - spacegroup 221
filenames_disp2_sc = ['sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp002.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid2_disp002.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp002.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp002.tar.gz']

# fcc - spacegroup 225
filenames_disp2_fcc = ['fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz']

# diam - spacegroup 227
filenames_disp2_diam = ['diam/disp/A_cF8_227_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid1_disp002.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid3_disp002.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid4_disp002.tar.gz']
# bcc - spacegroup 229
filenames_disp2_bcc = ['bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid1_disp002.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid2_disp002.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid4_disp002.tar.gz']
                         
                         
desc_files_disp2_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp2_hcp]
desc_files_disp2_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp2_sc]
desc_files_disp2_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp2_fcc]
desc_files_disp2_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp2_diam]
desc_files_disp2_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp2_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_disp2_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_disp2_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_disp2_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_disp2_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_disp2_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp002.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_c

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp002.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp002.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

In [237]:
structure_list_2 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_disp2 = []
for structure in structure_list_2:
    y_pred_disp2.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Displacements 4.0%

In [240]:
# hcp - spacegroup 194
filenames_disp4_hcp = ['hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid1_disp004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp004.tar.gz',
                        'hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid4_disp004.tar.gz']
# sc - spacegroup 221
filenames_disp4_sc = ['sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid2_disp004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp004.tar.gz',
                       'sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp004.tar.gz']

# fcc - spacegroup 225
filenames_disp4_fcc = ['fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                         'fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz']

# diam - spacegroup 227
filenames_disp4_diam = ['diam/disp/A_cF8_227_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid1_disp004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid3_disp004.tar.gz',
                          'diam/disp/A_cF8_227_a_target_nb_atoms128_rotid4_disp004.tar.gz']
# bcc - spacegroup 229
filenames_disp4_bcc = ['bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid1_disp004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid2_disp004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz',
                         'bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid4_disp004.tar.gz']
                         
                         
desc_files_disp4_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp4_hcp]
desc_files_disp4_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp4_sc]
desc_files_disp4_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp4_fcc]
desc_files_disp4_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp4_diam]
desc_files_disp4_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_disp4_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_disp4_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_disp4_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_disp4_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_disp4_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_disp4_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_c_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/disp/A_hP2_194_

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid1_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_target_nb_atoms128_rotid4_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/disp/A_cP1_221_a_targe

INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/fcc/disp/A_cF4_225_

INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_cF8_227_a_target_nb_atoms128_rotid2_disp004.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/diam/disp/A_c

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid0_disp004.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_a_target_nb_atoms128_rotid3_disp004.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/bcc/disp/A_cI2_229_

In [241]:
structure_list_4 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_disp4 = []
for structure in structure_list_4:
    y_pred_disp4.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Vacancies 1.0%

In [4]:
# hcp - spacegroup 194
filenames_vac1_hcp = ['hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac01.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac01.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac01.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac01.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac01.tar.gz']
# sc - spacegroup 221
filenames_vac1_sc = ['sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid1_vac01.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid2_vac01.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid3_vac01.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid4_vac01.tar.gz']

# fcc - spacegroup 225
filenames_vac1_fcc = ['fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac01.tar.gz']

# diam - spacegroup 227
filenames_vac1_diam = ['diam/vac/A_cF8_227_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid1_vac01.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid2_vac01.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid3_vac01.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid4_vac01.tar.gz']
# bcc - spacegroup 229
filenames_vac1_bcc = ['bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid0_vac01.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid1_vac01.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid2_vac01.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid3_vac01.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid4_vac01.tar.gz']
                         
                         
desc_files_vac1_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac1_hcp]
desc_files_vac1_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac1_sc]
desc_files_vac1_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac1_fcc]
desc_files_vac1_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac1_diam]
desc_files_vac1_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac1_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_vac1_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_vac1_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_vac1_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_vac1_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_vac1_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac01.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac01.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac01.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac01.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac01.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac01.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_

In [5]:
structure_list_vac1 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_vac1 = []
for structure in structure_list_vac1:
    y_pred_vac1.append(ase_get_spacegroup(structure, symprec=1e-1).no)

KeyboardInterrupt: 

## Vacancies 25%

In [6]:
# hcp - spacegroup 194
filenames_vac25_hcp = ['hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac25.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac25.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac25.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac25.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac25.tar.gz']
# sc - spacegroup 221
filenames_vac25_sc = ['sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid1_vac25.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid2_vac25.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid3_vac25.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid4_vac25.tar.gz']

# fcc - spacegroup 225
filenames_vac25_fcc = ['fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac25.tar.gz']

# diam - spacegroup 227
filenames_vac25_diam = ['diam/vac/A_cF8_227_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid1_vac25.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid2_vac25.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid3_vac25.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid4_vac25.tar.gz']
# bcc - spacegroup 229
filenames_vac25_bcc = ['bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid0_vac25.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid1_vac25.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid2_vac25.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid3_vac25.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid4_vac25.tar.gz']
                         
                         
desc_files_vac25_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac25_hcp]
desc_files_vac25_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac25_sc]
desc_files_vac25_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac25_fcc]
desc_files_vac25_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac25_diam]
desc_files_vac25_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac25_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_vac25_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_vac25_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_vac25_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_vac25_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_vac25_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac25.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac25.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac25.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac25.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac25.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac25.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_

In [ ]:
structure_list_vac25 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_vac25 = []
for structure in structure_list_vac25:
    y_pred_vac25.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Vacancies 50%

In [7]:
# hcp - spacegroup 194
filenames_vac50_hcp = ['hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac50.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac50.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac50.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac50.tar.gz',
                        'hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac50.tar.gz']
# sc - spacegroup 221
filenames_vac50_sc = ['sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid1_vac50.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid2_vac50.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid3_vac50.tar.gz',
                       'sc/vac/A_cP1_221_a_target_nb_atoms128_rotid4_vac50.tar.gz']

# fcc - spacegroup 225
filenames_vac50_fcc = ['fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                         'fcc/vac/A_cF4_225_a_target_nb_atoms128_rotid0_vac50.tar.gz']

# diam - spacegroup 227
filenames_vac50_diam = ['diam/vac/A_cF8_227_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid1_vac50.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid2_vac50.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid3_vac50.tar.gz',
                          'diam/vac/A_cF8_227_a_target_nb_atoms128_rotid4_vac50.tar.gz']
# bcc - spacegroup 229
filenames_vac50_bcc = ['bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid0_vac50.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid1_vac50.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid2_vac50.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid3_vac50.tar.gz',
                         'bcc/vac/A_cI2_229_a_target_nb_atoms128_rotid4_vac50.tar.gz']
                         
                         
desc_files_vac50_hcp = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac50_hcp]
desc_files_vac50_sc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac50_sc]
desc_files_vac50_fcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac50_fcc]
desc_files_vac50_diam = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac50_diam]
desc_files_vac50_bcc = [os.path.join(configs['io']['desc_folder'], item) for item in filenames_vac50_bcc]

y_true = []
_, structure_list_hcp = load_descriptor(desc_files=desc_files_vac50_hcp, configs=configs)
y_true = y_true + [194]*len(structure_list_hcp)

_, structure_list_sc = load_descriptor(desc_files=desc_files_vac50_sc, configs=configs)
y_true = y_true + [221]*len(structure_list_sc)

_, structure_list_fcc = load_descriptor(desc_files=desc_files_vac50_fcc, configs=configs)
y_true = y_true + [225]*len(structure_list_fcc)

_, structure_list_diam = load_descriptor(desc_files=desc_files_vac50_diam, configs=configs)
y_true = y_true + [227]*len(structure_list_diam)

_, structure_list_bcc = load_descriptor(desc_files=desc_files_vac50_bcc, configs=configs)
y_true = y_true + [229]*len(structure_list_bcc)

INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid0_vac50.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid1_vac50.tar.gz
INFO: Extracting file 3/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid2_vac50.tar.gz
INFO: Extracting file 4/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid3_vac50.tar.gz
INFO: Extracting file 5/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/hcp/vac/A_hP2_194_c_target_nb_atoms128_rotid4_vac50.tar.gz
INFO: Extracting file 1/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_rotid0_vac50.tar.gz
INFO: Extracting file 2/5: /home/ziletti/Documents/calc_nomadml/rot_inv_3d/desc_folder/sc/vac/A_cP1_221_a_target_nb_atoms128_

In [ ]:
structure_list_vac50 = structure_list_hcp + structure_list_sc + structure_list_fcc + structure_list_diam + structure_list_bcc

y_pred_vac50 = []
for structure in structure_list_vac25:
    y_pred_vac50.append(ase_get_spacegroup(structure, symprec=1e-1).no)

## Classification results

In [231]:
accuracy_score(y_true, y_pred_disp04)

0.0

In [232]:
accuracy_score(y_true, y_pred_disp1)

0.0

In [239]:
accuracy_score(y_true, y_pred_disp2)

0.0

In [243]:
accuracy_score(y_true, y_pred_disp4)

0.0

In [ ]:
accuracy_score(y_true, y_pred_vac1)

In [ ]:
accuracy_score(y_true, y_pred_vac25)

In [ ]:
accuracy_score(y_true, y_pred_vac50)

In [201]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)
cnf_matrix

array([[  0,   0],
       [900,   0]])

## Loading prototype database and check the classification for pristine structures

In [171]:
from ase.visualize import view
idx=0
# change idx if you want to visualize another structure 
# idx=0 visualizes the first structure in the list
# Note: 0<=idx<len(ase_atoms_list)
view(structure*(1, 1, 1), viewer='x3d')

In [128]:
db_files_prototypes_basedir = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/db_ase/'

proto_names = ['A_cP1_221_a']

prototypes_basedir = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/prototypes_aflow_new'
ase_db_files = [os.path.join(db_files_prototypes_basedir, proto_name) + '.db' for proto_name in proto_names]

db_protos = zip(proto_names, ase_db_files)

In [146]:
from ai4materials.utils.utils_data_retrieval import read_ase_db
from ai4materials.utils.utils_crystals import create_supercell
import random 

for idx_db, db_proto in enumerate(db_protos):
    ase_atoms_list = read_ase_db(db_path=ase_db_files[idx_db])

In [150]:
y_pred = []
y_pred_sc = []
for idx, structure in enumerate(ase_atoms_list[:4]):
    print idx
    supercell =  create_supercell(structure, create_replicas_by='nb_atoms', target_nb_atoms=128, optimal_supercell=True)
    alpha = random.random() * 360.0
    structure.rotate(alpha, 'x', rotate_cell=True, center='COU')

    beta = random.random() * 360.0
    structure.rotate(beta, 'y', rotate_cell=True, center='COU')

    gamma = random.random() * 360.0
    structure.rotate(gamma, 'z', rotate_cell=True, center='COU')
        
    y_pred.append(ase_get_spacegroup(structure, symprec=1e-1).no)
    y_pred_sc.append(ase_get_spacegroup(supercell, symprec=1e-1).no)

0
1
2
3


In [154]:
y_true = [221]*len(ase_atoms_list)
accuracy_score(y_pred, y_pred_sc)

1.0